# Querying online data with astroquery and PyVO

There are two main general packages for accessing online data from Python in the Astropy ecosystem:

* The [astroquery](https://astroquery.readthedocs.io/en/latest/) coordinated package, which offers access to many services, including a number that are not VO compatible.
* The [PyVO](https://pyvo.readthedocs.io/en/latest/) affiliated package which implements a Pythonic interface to VO-compliant services.

In this tutorial, we will take a look at both of these.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Querying services such as Simbad and ESASky</li>
<li>Using PyVO to access data on VO-compliant servers</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astroquery and PyVO. For more information about the features presented below as well as other available features, you can read the
[astroquery](https://astroquery.readthedocs.io/en/latest/) and [PyVO](https://pyvo.readthedocs.io/en/latest/) documentation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Using astroquery

Astroquery provides a common interface to the following services:

* ALMA Queries (astroquery.alma)
* Atomic Line List (astroquery.atomic)
* Besancon Queries (astroquery.besancon)
* Cadc (astroquery.cadc)
* CASDA Queries (astroquery.casda)
* CDS MOC Service (astroquery.cds)
* esa.hubble (astroquery.esa.hubble)
* ESASky Queries (astroquery.esasky)
* ESO Queries (astroquery.eso)
* Gaia TAP+ (astroquery.gaia)
* GAMA Queries (astroquery.gama)
* HEASARC Queries (astroquery.heasarc)
* HITRAN Queries (astroquery.hitran)
* IRSA Image Server program interface (IBE) Queries (astroquery.ibe)
* IRSA Queries (astroquery.irsa)
* IRSA Dust Extinction Service Queries (astroquery.irsa_dust)
* JPL Spectroscopy Queries (astroquery.jplspec)
* MAGPIS Queries (astroquery.magpis)
* MAST Queries (astroquery.mast)
* Minor Planet Center Queries (astroquery.mpc/astroquery.solarsystem.MPC)
* NASA ADS Queries (astroquery.nasa_ads)
* NED Queries (astroquery.ned)
* NIST Queries (astroquery.nist)
* NRAO Queries (astroquery.nrao)
* NVAS Queries (astroquery.nvas)
* SIMBAD Queries (astroquery.simbad)
* Skyview Queries (astroquery.skyview)
* Splatalogue Queries (astroquery.splatalogue)
* UKIDSS Queries (astroquery.ukidss)
* Vamdc Queries (astroquery.vamdc)
* VizieR Queries (astroquery.vizier)
* VO Simple Cone Search (astroquery.vo_conesearch)
* VSA Queries (astroquery.vsa)
* xMatch Queries (astroquery.xmatch)

and also provides access to other services.

### Simbad

To start off, we can take a look at the sub-package to query [SIMBAD](http://simbad.u-strasbg.fr/simbad/):

In [ ]:
from astroquery.simbad import Simbad

We can query by identifier:

In [ ]:
result = Simbad.query_object("m1")
result

or by coordinates:

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
c = SkyCoord(5 * u.hourangle, 30 * u.deg, frame='icrs')
r = 15 * u.arcminute

In [ ]:
result = Simbad.query_region(c, radius=r)
result

### ESASky

Another example is querying images from [ESASky](https://astroquery.readthedocs.io/en/latest/esasky/esasky.html)

In [ ]:
from astroquery.esasky import ESASky

We can list the available catalogs and maps:

In [ ]:
ESASky.list_catalogs()

In [ ]:
ESASky.list_maps()

and we can query and download catalogs:

In [ ]:
from astroquery.esasky import ESASky
result = ESASky.query_object_catalogs("M51", "2MASS")
result[0]

In [ ]:
result = ESASky.query_region_catalogs("M51", 10 * u.arcmin, "2MASS")
result[0]

We can also query and download images:

In [ ]:
images = ESASky.get_images("m51", radius=5 * u.arcmin,
                           missions=['Herschel'])

In [ ]:
images['HERSCHEL'][0]['160'].info()

In [ ]:
plt.imshow(images['HERSCHEL'][0]['160']['image'].data)


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Take a look at the documntation for <a href="https://astroquery.readthedocs.io/en/latest/irsa/irsa.html">IRSA queries</a> and try and download a table from the ALLWISE Source Catalog by searching for sources within 5 arcminutes of M31.</p>

</div>

</section>


In [ ]:
from astroquery.irsa import Irsa
Irsa.list_catalogs()  # shows that catalog name is allwise_p3as_psd
table = Irsa.query_region("M31", catalog="allwise_p3as_psd", radius=5 * u.arcmin)
table

## Using PyVO

The PyVO package differs a bit from astroquery in that it does not have specialized sub-packages for different services. Instead, it implements and exposes the VO query standards. It is a lower level package that may require more knowledge about the VO, so it is not necessarily as user friendly as astroquery, but on the other hand it can work with any VO-compliant service. Astroquery is starting to use PyVO in places behind the scenes.

We can take a look at an example that consists of downloading the 2MASS images for the M17 region:

In [ ]:
from pyvo.dal import imagesearch

In [ ]:
M17 = SkyCoord.from_name('M17')
table = imagesearch('https://irsa.ipac.caltech.edu/cgi-bin/2MASS/IM/nph-im_sia?type=at&ds=asky&',
                    M17, size=0.25).to_table()
table

In [ ]:
subtable = table[(table['band'] == b'K') & (table['format'] == b'image/fits')]
subtable

In [ ]:
from astropy.io import fits
hdus = [fits.open(row['download'].decode('ascii')) for row in subtable]

In [ ]:
plt.imshow(hdus[0][0].data, vmin=500, vmax=1000)

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)